In [1]:
# importing required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# reading the data
data = pd.read_csv('nyc_taxi_trip_duration.csv')

In [3]:
# first five rows of the data
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,N,400
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,N,1100
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,N,1635
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,N,1141
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,N,848


In [4]:
# columns in the dataset
data.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration'],
      dtype='object')

## Description of the variables


<b>id</b> : a unique identifier for each trip. This is a <b>nominal</b> data column.

<b>vendor_id</b> :  a code indicating the provider associated with the trip record.This is a <b>nominal</b> data column.

<b>pickup_datetime</b> : the date and the time when the ride started.

<b>dropoff_datetime</b> : the date and time when the ride ended

<b>passenger_count</b> : the number of passengers in the vehicle (driver entered value)

<b>pickup_longitude</b> : date and time when the meter was engaged

<b>pickup_latitude</b> : date and time when the meter was disengaged

<b>dropoff_longitude</b> : the longitude where the meter was disengaged

<b>dropoff_latitude</b> : the latitude where the meter was disengaged

<b>store_and_fwd_flag</b> : This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server (Y=store and forward; N=not a store and forward trip).This column is categorical

<b>trip_duration</b> :  (target) duration of the trip in seconds

In [5]:
# shape of the data
data.shape

(729322, 11)

In [6]:
# statistics of the numerical features in the data
data.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,729322.000000,729322.000000,729322.000000,729322.000000,729322.000000,729322.000000,7.293220e+05
mean,1.535403,1.662055,-73.973513,40.750919,-73.973422,40.751775,9.522291e+02
std,0.498745,1.312446,0.069754,0.033594,0.069588,0.036037,3.864626e+03
min,1.000000,0.000000,-121.933342,34.712234,-121.933304,32.181141,1.000000e+00
25%,1.000000,1.000000,-73.991859,40.737335,-73.991318,40.735931,3.970000e+02
50%,2.000000,1.000000,-73.981758,40.754070,-73.979759,40.754509,6.630000e+02
75%,2.000000,2.000000,-73.967361,40.768314,-73.963036,40.769741,1.075000e+03
max,2.000000,9.000000,-65.897385,51.881084,-65.897385,43.921028,1.939736e+06


In [7]:
# overview of type of variables
data.dtypes

id                     object
vendor_id               int64
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
dtype: object

In [7]:
# checking the missing values in the dataset
data.isnull().sum()

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

In [9]:
# looking at the longest trips
print('Longest 5 trip duration: \n {} '.format(data['trip_duration'].nlargest(5)))
print('\nThe the number of rows with 0 as their trip duration values is {}'.format(len(data[data['trip_duration']==1 ])))

Longest 5 trip duration: 
 21813     1939736
259437      86391
119185      86387
177225      86378
496391      86377
Name: trip_duration, dtype: int64 

The the number of rows with 0 as their trip duration values is 13


In [9]:
# There is 1 record with extremely large value of 1939736 and 13 with 0 seconds each. We can drop these rows. 

In [10]:
# dropping the outliers
data=data[data.trip_duration!=data.trip_duration.max()]
data=data[data.trip_duration!=data.trip_duration.min()]

In [11]:
# converting the date time variables to datatime format
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])

In [12]:
# creating datetime features
data['pickup_day']=data['pickup_datetime'].dt.day_name()
data['dropoff_day']=data['dropoff_datetime'].dt.day_name()
data['pickup_month']=data['pickup_datetime'].dt.month
data['dropoff_month']=data['dropoff_datetime'].dt.month

In [13]:
# columns of the data
data.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration', 'pickup_day', 'dropoff_day', 'pickup_month',
       'dropoff_month'],
      dtype='object')

In [19]:
# dropping the variables which might not be helpful to predict the trip duration
data = data.dropna(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude'], axis=1)

C:\Users\satha\AppData\Local\Temp\ipykernel_2880\4225863575.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  data = data.dropna(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime', 'pickup_longitude', 'pickup_latitude',


TypeError: DataFrame.dropna() got multiple values for argument 'axis'

In [23]:
# converting the categorical variables to numerical variables
data = pd.get_dummies(data)
print(data)
print(data.shape)

        passenger_count  trip_duration  pickup_month  dropoff_month  \
0                     1            400             2              2   
1                     2           1100             3              3   
2                     2           1635             2              2   
3                     6           1141             1              1   
4                     1            848             2              2   
...                 ...            ...           ...            ...   
729317                2            296             5              5   
729318                1            315             2              2   
729319                1            673             4              4   
729320                1            447             6              6   
729321                4           1224             1              1   

        store_and_fwd_flag_N  store_and_fwd_flag_Y  pickup_day_Friday  \
0                          1                     0                  0   
1

In [25]:
# separating dependent and independent variables
X = data.drop(['trip_duration'], 1)
print(X)
y = data['trip_duration']
print(y)

        passenger_count  pickup_month  dropoff_month  store_and_fwd_flag_N  \
0                     1             2              2                     1   
1                     2             3              3                     1   
2                     2             2              2                     1   
3                     6             1              1                     1   
4                     1             2              2                     1   
...                 ...           ...            ...                   ...   
729317                2             5              5                     1   
729318                1             2              2                     1   
729319                1             4              4                     1   
729320                1             6              6                     1   
729321                4             1              1                     1   

        store_and_fwd_flag_Y  pickup_day_Friday  pickup_day_Mon

C:\Users\satha\AppData\Local\Temp\ipykernel_2880\2863005183.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = data.drop(['trip_duration'], 1)


In [27]:
# creating a training and validation set
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.25, random_state=10)
print(x_train,x_valid,y_train,y_valid)

        passenger_count  pickup_month  dropoff_month  store_and_fwd_flag_N  \
456760                2             5              5                     1   
381734                1             3              3                     1   
169486                2             6              6                     1   
487051                2             5              5                     1   
179121                1             6              6                     1   
...                 ...           ...            ...                   ...   
533746                1             3              3                     1   
105610                1             3              3                     1   
617939                1             1              1                     1   
443778                1             1              1                     1   
345396                1             4              4                     1   

        store_and_fwd_flag_Y  pickup_day_Friday  pickup_day_Mon

## Linear Regression Model

In [29]:
# creating the model
lreg = LinearRegression()

In [30]:
# training the model
lreg.fit(x_train, y_train)

LinearRegression()

In [31]:
# rmse on training set
pred_train = lreg.predict(x_train)
rmse_train = np.sqrt(mean_squared_error(y_train, pred_train))
rmse_train

3095.259185666213

In [32]:
# rmse on validation set
pred_val = lreg.predict(x_valid)
rmse = np.sqrt(mean_squared_error(y_valid, pred_val))
rmse

3145.819342129174